In [1]:
# DON'T RUN THIS BLOCK MORE THAN ONCE
import sys, os
from pathlib import Path

nrpy_dir_path = Path.cwd().parent
if str(nrpy_dir_path) not in sys.path:
    sys.path.append(str(nrpy_dir_path))

os.chdir(nrpy_dir_path)
print(nrpy_dir_path)

/media/goncalo/HDD/TESE/nrpytutorial


In [2]:
# Load important packages/modules
import sympy as sp
# from pathlib import Path
import shutil, time, os
import finite_difference as fin
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
from outputC import *
import reference_metric as rfm

# Create directories for:
# 1. C codes generated by NRPy+
module = Path.cwd() / 'ScalarFieldStaticBackground'
Cdir = module / 'ScalarFieldPlayground'
# 2. Output files and executables
outdir = Cdir / 'output'
errdir = outdir / 'err'
toutdir = outdir / 'out'
# Main C file
# mains = Cdir.glob('main*.c')
# for file in mains:
#     shutil.copy(str(file), str(module))
# if main.exists():
#     shutil.copy(str(main), str(module))
# Sbatch folder
sbatch = Cdir / 'sbatch'
#backup_sbatch = module / 'sbatch'
#if sbatch.exists():
#    for file in sbatch.glob('*.sbatch'):
#        shutil.copy(str(file), str(backup_sbatch))
# Remove the main directory if it exists
shutil.rmtree(Cdir, ignore_errors=True)
# Create all directories
Cdir.mkdir()
outdir.mkdir()
errdir.mkdir()
toutdir.mkdir()
# Copy backup main.c file to Cdir
backupmains = module.glob('main*.c')
for file in backupmains:
    shutil.copy(str(file), str(Cdir))
# Copy Makefile
makefile = module / 'Makefile'
shutil.copy(str(makefile), str(Cdir))
# backupmain = module / 'main.c'
# shutil.copy(str(backupmain), str(Cdir))
# Copy backup sbatch files for baltasar
sbatch.mkdir()
#for file in backup_sbatch.glob('*.sbatch'):
#    shutil.copy(str(file), str(sbatch))

In [3]:
# Set tpatial dimension to 3
DIM = 3
par.set_parval_from_str('grid::DIM', DIM)

# Set coordinate system
CoordSystem = 'Spherical'
par.set_parval_from_str('reference_metric::CoordSystem', CoordSystem)

# Set boundary condition type
# QPE -> Quadratic Polynomial Extrapolation
# Sommerfeld -> Outgoing radiation 
BC_Type = 'Sommerfeld'

# Set domain size and width for SinhSpherical coordinates
domain_size = 128
sinh_width = 0.2

# Choose the Runge-Kutta method employed in the MoL evolution
RK_method = 'RK4'

# Set the finite difference order (even numbers only)
FD_order = 4
par.set_parval_from_str('finite_difference::FD_CENTDERIVS_ORDER', FD_order)

# Set number of ghost cells from the finite difference order
NGHOSTS = int(FD_order / 2) + 1

# Default type for REAL numbers
REAL = 'double'

# Default CFL Factor
# GETS OVERWRITTEN WHEN EXECUTED 
# In pure axisymmetry (symmetry_axes = 2 below) 1.0 works fine. Otherwise 0.5 or lower.
default_CFL_FACTOR = 0.5 

# Copy SIMD/SIMD_intrinsics.h to $Cdir/SIMD/SIMD_intrinsics.h
SIMDdir = Cdir / 'SIMD'
SIMDdir.mkdir()
shutil.copy(str(nrpy_dir_path / 'SIMD' / 'SIMD_intrinsics.h'), str(Cdir / 'SIMD'))

# Set axisymetry, i.e. all derivatives along phi (axis 2) are set to zero
# par.set_parval_from_str("indexedexp::symmetry_axes","2")

# Make sbatch files?
make_sbatch = False

In [4]:
# Import module to create the initial data for a scalar field in flat space
import ScalarFieldStaticBackground.scalar_field_ID as sfid

# Set type of initial data to create
IDStateType = 'SphericalGaussian'
par.set_parval_from_str('ScalarFieldStaticBackground.scalar_field_ID::IDStateType', IDStateType)

# Function to export the C code for the initial data
def InitialDataC():
    print('Generating optimized C code for a scalar field in flat space. May take a while, depending on the coordinate system...')
    start = time.time()

    # Create initial data and write it to the C function dictionary
    sfid.scalar_field_ID()
    # Write the function in a C file
    with open(Cdir / 'initial_data.h', 'w') as file:
        file.write(outC_function_dict['initial_data'])

    end = time.time()
    print('Generated code for the initial data in ' + str(end - start) + ' seconds!')

# Function to export the C code for the initial data
def TestInitialDataC():
    print('Generating optimized C code for a scalar field in flat space. May take a while, depending on the coordinate system...')
    start = time.time()

    # Create initial data and write it to the C function dictionary
    sfid.scalar_field_test_ID()
    # print(sfid.alpha)
    # print(sfid.betaU)
    # print(sfid.BU)
    print(sfid.Phi)
    print(sfid.Pi)
    # Write the function in a C file
    with open(Cdir / 'initial_data.h', 'w') as file:
        file.write(outC_function_dict['initial_data'])

    end = time.time()
    print('Generated code for the initial data in ' + str(end - start) + ' seconds!')

In [5]:
TestInitialDataC()

Generating optimized C code for a scalar field in flat space. May take a while, depending on the coordinate system...
exp(-(r0 + xx0)**2/w**2)/(2*xx0) + exp(-(-r0 + xx0)**2/w**2)/(2*xx0)
-(-(-2*r0 - 2*xx0)*exp(-(r0 + xx0)**2/w**2)/(2*w**2) - (-2*r0 + 2*xx0)*exp(-(-r0 + xx0)**2/w**2)/(2*w**2))/xx0
Generated code for the initial data in 0.6729238033294678 seconds!


In [6]:
# Import modules for setting up evolution equations
import ScalarFieldStaticBackground.scalar_field_RHS as sfrhs
import ScalarFieldStaticBackground.gauge_RHS as grhs
import ScalarFieldStaticBackground.gridfunctions_and_metric_quantities as gmq

print('Generating symbolic expression for the RHSs of the evolution equations...')
start = time.time()

# Set the gauge evolution options
par.set_parval_from_str('ScalarFieldStaticBackground.gauge_RHS::LapseEvolutionOption', 'OnePlusLog')
par.set_parval_from_str('ScalarFieldStaticBackground.gauge_RHS::ShiftEvolutionOption', 'GammaDriving2ndOrder_Covariant')

# We are not enabling rfm_precompute because the evolution equations
# because in this formulation the hatted quantities that benefit from precomputation
# are not used to establish the RHS expressions

# We don't need to set the evolved conformal factor type because cf is not evolved in this framework

# Generate the RHS expressions
sfrhs.field_RHSs()
# grhs.gauge_RHSs()

# Define the upwinding control vector
betaU = gmq.betaU

# Don't need to enforce the constraint on the determinant of \bar{\gamma} because it won't be evolved

# FINISHED
end = time.time()
print('Finished generating symbolic RHS expressions for the evolution equations in ' + str(end - start) + ' seconds!')

Generating symbolic expression for the RHSs of the evolution equations...
Finished generating symbolic RHS expressions for the evolution equations in 0.06109142303466797 seconds!


In [7]:
# Create function to export RHS expressions to C code
def RHSs():
    print('Generating C code for RHS expressions in ' + CoordSystem + ' coordinates.')
    start = time.time()

    # Construct the LHSs and RHS expressions for all evolved quantities
    lhs_names = [     'Phi',         'Pi']
    rhs_exprs = [sfrhs.Phi_rhs, sfrhs.Pi_rhs]
    # for i in range(DIM):
    #     lhs_names.append(    'betU' + str(i))
    #     rhs_exprs.append(grhs.bet_rhsU[i])
    #     lhs_names.append(    'vetU' + str(i))
    #     rhs_exprs.append(grhs.vet_rhsU[i])

    # Sort the lhss list alphabetically, and rhss to match.
    #   This ensures the RHSs are evaluated in the same order 
    #   they're allocated in memory:
    # lhs_names,rhs_exprs = [list(x) for x in zip(*sorted(zip(lhs_names,rhs_exprs), key=lambda pair: pair[0]))]

    # Declare the list of lhrh's
    evol_rhss = []
    for var in range(len(lhs_names)):
        evol_rhss.append(lhrh(lhs=gri.gfaccess('rhs_gfs', lhs_names[var]), rhs=rhs_exprs[var]))

    # SEt up the C function for the RHSs
    desc = 'Evaluate the RHSs'
    name = 'rhs_eval'
    outCfunction(
        outfile = os.path.join(Cdir, name + '.h'),
        desc = desc,
        name = name,
        params = '''const paramstruct *restrict params, REAL ** xx, const REAL *restrict in_gfs, REAL *restrict rhs_gfs''',
        body = fin.FD_outputC('returnstring', evol_rhss, params='outCverbose=False,SIMD_enable=False',
                                upwindcontrolvec=betaU).replace('IDX4', 'IDX4S'),
        loopopts = 'InteriorPoints,Read_xxs')
    end = time.time()
    print('Finished RHS C codegen in ' + str(end - start) + ' seconds!')

In [8]:
RHSs()

Generating C code for RHS expressions in Spherical coordinates.
Output C function rhs_eval() to file /media/goncalo/HDD/TESE/nrpytutorial/ScalarFieldStaticBackground/ScalarFieldPlayground/rhs_eval.h
Finished RHS C codegen in 0.10085058212280273 seconds!


In [9]:
# Apply curvilinear boundary conditions
import CurviBoundaryConditions.CurviBoundaryConditions as cbcs
cbcs.Set_up_CurviBoundaryConditions(os.path.join(Cdir / 'boundary_conditions'), BoundaryCondition=BC_Type)

if BC_Type == "Sommerfeld":
    bcs = cbcs.sommerfeld_bc(vars_radpower_default=3., vars_speed_default=1., vars_at_inf_default=0.)
    
    # We could change values of GF speed or values at infinity, but the defaults work for now
    # Write values to parameter file
    bcs.write_to_sommerfeld_params_file(Cdir)

Wrote to file "/media/goncalo/HDD/TESE/nrpytutorial/ScalarFieldStaticBackground/ScalarFieldPlayground/boundary_conditions/parity_conditions_symbolic_dot_products.h"
Evolved parity: ( Phi:0, Pi:0 )


Wrote to file "/media/goncalo/HDD/TESE/nrpytutorial/ScalarFieldStaticBackground/ScalarFieldPlayground/boundary_conditions/EigenCoord_Cart_to_xx.h"


In [10]:
# Generate timestepping code
import MoLtimestepping.C_Code_Generation as MoL

MoLdir = Cdir / 'MoLtimestepping'
MoLdir.mkdir()

if BC_Type == "QPE":
    RHS_string = 'rhs_eval(&params, xx, RK_INPUT_GFS, RK_OUTPUT_GFS);'
    post_RHS_string = 'apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_OUTPUT_GFS);'
elif BC_Type == "Sommerfeld":
    RHS_string = '''rhs_eval(&params, xx, RK_INPUT_GFS, RK_OUTPUT_GFS);
apply_bcs_sommerfeld(&params, xx, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_INPUT_GFS, RK_OUTPUT_GFS);'''
    post_RHS_string = ''

MoL.MoL_C_Code_Generation(RK_method,
    RHS_string = RHS_string,
    post_RHS_string = post_RHS_string,
    outdir = MoLdir)

In [11]:
# Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
# DOESN'T NEED TO BE REPEATED, AT LEAST IN THIS CASE
# par.generate_Cparameters_Ccodes(os.path.join(Cdir))

# Set free_parameters.h

# Following the Black Hole collision tutorial, one could set eta freely 
# by manually writing in the free_parameters.h file.
# For now, let us go with the default value of eta = 2.0.

# Append to $Ccodesdir/free_parameters.h reference metric parameters based on generic
#    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
#    parameters set above. 
rfm.out_default_free_parameters_for_rfm(os.path.join(Cdir, "free_parameters.h"), domain_size, sinh_width)

# Step 3.e.iii: Generate set_Nxx_dxx_invdx_params__and__xx.h:
rfm.set_Nxx_dxx_invdx_params__and__xx_h(Cdir)

# Step 3.e.iv: Generate xxCart.h, which contains xxCart() for
#               (the mapping from xx->Cartesian) for the chosen
#               CoordSystem:
rfm.xxCart_h("xxCart", "./set_Cparameters.h", os.path.join(Cdir, "xxCart.h"))

# We have to run generate_Cparameters_Ccodes again because if we don't, RMAX doesn't get declared
# I don't know if there are any other differences

# Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(Cdir))

In [12]:
# Code to define REAL, NGHOSTS and CFL_FACTOR
with open(Cdir / 'REAL__NGHOSTS__CFL_FACTOR.h', 'w') as file:
    file.write('''
// Part P0.a: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
#define NGHOSTS {}
// Part P0.b: Set the numerical precision (REAL) to double, ensuring all floating point
//            numbers are stored to at least ~16 significant digits
#define REAL {}
// Part P0.c: Set the CFL Factor. Can be overwritten at command line.
REAL CFL_FACTOR = {};'''.format(NGHOSTS, REAL, default_CFL_FACTOR))

In [13]:
# Output code to find the timestep according to the CFL factor and computational grid
rfm.out_timestep_func_to_file(Cdir / 'find_timestep.h')

In [14]:
from pathlib import Path
import cmdline_helper as cmd

def make_batch(batch_path, exec_dir, params, props):
    
    # mandatory keys in props: 
    # job_name, output, error, ntasks, mem
    
    # optional keys in props with default values:
    # nodes, mail_user, mail_type, time
    if 'nodes' not in props.keys():
        props['nodes'] = 1
    if 'mail_user' not in props.keys():
        props['mail_user'] = 'goncalo.j.c.andrade@tecnico.ulisboa.pt'
    if 'mail_type' not in props.keys():
        props['mail_type'] = 'ALL'
    if 'time' not in props.keys():
        props['time'] = '72:00:00'
        
    batch_props = """#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --output={output}
#SBATCH --error={error}
#SBATCH --nodes={nodes}
#SBATCH --ntasks={ntasks}
#SBATCH --mail-user={mail_user}
#SBATCH --mail-type={mail_type}
#SBATCH --time={time}
#SBATCH --mem={mem}\n\n""".format_map(props)
    
    batch_exec_dir = """RUNPATH={} ; cd $RUNPATH\n""".format(exec_dir)
    
    mkdir = f"mkdir {props['job_name']}_{params['nr']}_{params['nt']}_{params['np']} ; cd {props['job_name']}_{params['nr']}_{params['nt']}_{params['np']}\n"
    
    batch_omp = "export OMP_NUM_THREADS={ntasks}\n".format_map(props)
    
    batch_run_cmd = "../{exec_name} {nr} {nt} {np} {cfl} {amp} {peak} {width} {mass}\n".format_map(params)
    
    with open(batch_path, 'w') as file:
        file.write(batch_props + batch_exec_dir + mkdir + batch_omp + batch_run_cmd)
    

if make_sbatch:
    
    exec_dir = Path('/home', 'goncaloa', 'StaticScalarField_Sommerfeld', 'output')

    props = {}
    props['job_name'] = 'sommerfeld_far'
    props['output'] = str(exec_dir) + '/out/{}_%j.out'.format(props['job_name'])
    props['error'] = str(exec_dir) + '/err/{}_%j.err'.format(props['job_name'])
    props['ntasks'] = 24
    props['mem'] = '32G'

    params = {'exec_name':'exec', 'cfl':0.5, 'amp':1.0, 'peak':32.0, 'width':8.0, 'mass':1.0}

    try:
        nrpy_dir_path
    except NameError:
        Cdir = Path.cwd() / 'ScalarFieldPlayground'
        outdir = Cdir / 'output'
        batchdir = Cdir / 'sbatch'
    else:
        batchdir = sbatch

    if not batchdir.exists():
        cmd.mkdir(batchdir)
        print('batchdir created...')
    else:
        print('batchdir already exists...')

    r_res = [128, 256, 512, 1024]
    th_res = [8, 16, 32]
    phi_res = [16, 32, 64]

    i = 0

    sbatch_str = ""

    for NR in r_res:
        for NT in th_res:
            for NP in phi_res:

                #if NT == 32 and NP == 16:
                #    continue

                sbatchname = 'job_{}_{}_{}.sbatch'.format(NR, NT, NP)
                sbatch_file = batchdir / sbatchname

                params['nr'] = NR
                params['nt'] = NT
                params['np'] = NP

                make_batch(sbatch_file, exec_dir, params, props)
                sbatch_str += 'sbatch {} ; '.format(sbatchname)
                i += 1

    print(sbatch_str)
    
else: print('make_sbatch set to False!')

make_sbatch set to False!
